# Knowledge Graph Construction
> Relation and entity extraction from text

In [ ]:
#| default_exp ml.kg.cons

In [ ]:
#| hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export
from typing import TypeAlias, Iterable, List, Set
import numpy as np

In [ ]:
#|export

Entity: TypeAlias = tuple[str, str]
Relation: TypeAlias = str
Triplet: TypeAlias = tuple[Entity, Relation, Entity]

In [ ]:
#|export

def evaluate_joint_er_extraction(*, reference: Iterable[Triplet], prediction: Iterable[Triplet]):
    """
    Example: [(('John', 'PERSON'), 'works_at', ('Google', 'ORG'))]
    """

    reference_set = set(reference)
    prediction_set = set(prediction)
    assert len(reference) == len(reference_set), "Duplicates found in references"

    TP = len(reference_set & prediction_set)
    FP = len(prediction_set - reference_set)
    FN = len(reference_set - prediction_set)
    
    # Calculate metrics
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1_score
    }

def evaluate_joint_er_extractions(*, references: Iterable[Iterable[Triplet]], predictions: Iterable[Iterable[Triplet]]):
    score_dicts = [
        evaluate_joint_er_extraction(reference=reference, prediction=prediction) 
        for reference, prediction in zip(references, predictions)
    ]
    return {('mean_' + key): np.mean([scores[key] for scores in score_dicts]) for key in score_dicts[0].keys()}

In [ ]:
reference = [(('John', 'PERSON'), 'works_at', ('Google', 'ORG')), (('Mike', 'PERSON'), 'lives_in', ('Paris', 'LOC')), (('Dwight', 'PERSON'), 'sells', ('Paper', 'OBJ'))]
prediction = [(('John', 'PERSON'), 'works_at', ('Google', 'ORG')), (('Mike', 'PERSON'), 'lives_in', ('New York', 'LOC'))]

scores = evaluate_joint_er_extraction(reference=reference, prediction=prediction)
test_eq(scores, {'precision': 0.5, 'recall': 0.3333333333333333, 'f1': 0.4})
print(scores)

{'precision': 0.5, 'recall': 0.3333333333333333, 'f1': 0.4}


In [ ]:
references = [
    [(('John', 'PERSON'), 'works_at', ('Google', 'ORG')), (('Mike', 'PERSON'), 'lives_in', ('Paris', 'LOC')), (('Dwight', 'PERSON'), 'sells', ('Paper', 'OBJ'))],
    [(('Henry', 'PERSON'), 'founded', ('Ford', 'ORG'))],
]
predictions = [
    [(('John', 'PERSON'), 'works_at', ('Google', 'ORG')), (('Mike', 'PERSON'), 'lives_in', ('New York', 'LOC'))],
    [(('Henry', 'PERSON'), 'founded', ('Boston Dynamics', 'ORG'))],
]

scores = evaluate_joint_er_extractions(references=references, predictions=predictions)
test_eq(scores, {'mean_precision': 0.25, 'mean_recall': 0.16666666666666666, 'mean_f1': 0.2})
print(scores)

{'mean_precision': 0.25, 'mean_recall': 0.16666666666666666, 'mean_f1': 0.2}


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()